## Imports and defs

In [1]:
from operator import le
import numpy as np
from numpy.fft import fft
import matplotlib.pyplot as plt
from os import read
import math
import pandas as pd
from scipy import signal

def toInt(in_array):
    tmp = []
    for i in range(len(in_array)):
        tmp.append(int(in_array[i]))
    return tmp
def fillWindow(w_size, value):
    window = []
    for x in range (w_size):
        window.append(value)
    return window

def getSlindingWindows(in_array, w_size):
    windows = []
    tmp_window = fillWindow(w_size, in_array[0])
    for x in range (len(in_array)):
        tmp_window = tmp_window[1:]
        tmp_window.append(in_array[x])
        windows.append(tmp_window)
    return windows



def getWindowsFromArrayOfSize(in_array, w_size):
    windows = []
    for x in range(math.floor(len(in_array) / w_size)):
        tmp = []
        for w in range(w_size):
            tmp.append(in_array[x*w_size + w])
        windows.append(tmp)
    return windows


    

def appendZeros(in_array, zeros):
    tmp = in_array
    for x in range(zeros):
        tmp.append(0)
    return tmp


def getAverage(window):
    sum = 0
    for x in range(0, len(window)):
        sum+= window[x]
    avg = sum / len(window)
    return avg



def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y


def getMin(window):
    min = 9000
    for x in range(0, len(window)):
        if(window[x] <  min):
            min = window[x]
    return min

def getMax(window):
    max = 0
    for x in range(0, len(window)):
        if(window[x] >  max):
            max = window[x]
    return max 





#TODO: we here
def fix_window(window, window_size):
    d = window[window_size - 1]
    for x in range (len(window) - window_size):
        window.append(d)

def get_n_above_threshold(window, threshold):
    n = 0
    for x in window:
        if x > threshold:
            n = n + 1
    return n

def firstDev(window):
    dev = 0
    devs = []
    p = 0
    for x in range (1, len(window)):
        dev = (window[x] - window[x-1])
        devs.append(dev)
    devs.append(dev)
    return devs


        

## File reading

In [6]:
window_size = 50
directory = 'data/10_dvfs/'


In [7]:
import os
data_ = []
for filename in os.listdir(directory):
    f_name = os.path.join(directory, filename)
    f = open(f_name, 'r')
    raw = []
    for l in range(4000):
        r = f.readline()
        raw.append(r)
    data_file = toInt(raw)
    data_ += data_file


filtered = data_
#filtered = butter_highpass_filter(data_, cut_freq, fs, order)


windows = getWindowsFromArrayOfSize(filtered, window_size)


devs = []
for w in windows:
    devs.append(firstDev(w))

print(len(data_))


40000


In [ ]:

df2 = pd.Series(data = filtered[:800])
df2 = pd.Series(data = devs[15])
df2.plot()

      #50 100 150 200 
#200 -  1  0   1   0 

## Decoding

In [8]:
packages = 100
bits = 8
avgs = []
bin = 1
max =  getAverage(devs[0])
avgs.append(max)
for w in range(1, packages * bits):
    avg = getAverage(devs[w])
    bin = (bin << 1) | (avg >= 0)
    avgs.append(avg)
    
   
print(hex(bin))


og = 0
for x in range(packages):
    og = (og << 8) | (0xaa)
print(hex(og))
print(hex(bin))

err = og ^ bin 
err_str = '{0:08b}'.format(err) 
err_count = err_str.count('1')

ber = 100* (err_count / (bits * packages))
print(round(ber, 2), "%")

0x8ad6d4b656d77695cb45da26a5ad2d6d4f0b3f76c6d6e6d492a5b7a4f18ce25a46929ed758ad279952d6969e96bdbd657517d6f534da96a4bd91b9b9a6b5b5b4b5797949373312d124b7252d2dad136a5ed256529adab5b532591697a5a7a52d2c4e31fd
0xaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
0x8ad6d4b656d77695cb45da26a5ad2d6d4f0b3f76c6d6e6d492a5b7a4f18ce25a46929ed758ad279952d6969e96bdbd657517d6f534da96a4bd91b9b9a6b5b5b4b5797949373312d124b7252d2dad136a5ed256529adab5b532591697a5a7a52d2c4e31fd
53.37 %
